In [1]:
from ragas.metrics import FactualCorrectness, Faithfulness, LLMContextPrecisionWithReference, LLMContextRecall, ResponseRelevancy, SemanticSimilarity
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

load_dotenv()
link = os.getenv('dsa_2214')
token = os.getenv('team_token')
os.environ['OPENAI_API_KEY'] = token

chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
actual_concepts = []
with open('data/sorting.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        # print(line.strip())
        words = line.strip().split('->')
        # print(concepts)
        for concept in words:
            if concept not in actual_concepts:
                actual_concepts.append(concept)

actual_concepts = [' '.join(actual_concepts)] * 4
# actual_concepts

In [3]:
from src.extractor import LLM_Relation_Extractor

# only considering chapters 6 - 10 (sorting and searching)
rs_extractor = LLM_Relation_Extractor(link, token, chapters[6:10], 'bibliography')

<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


In [4]:
concepts = rs_extractor.identify_concepts()

In [5]:
sorting_concepts = {}

start = rs_extractor.chapters.index('Internal Sorting')
end = rs_extractor.chapters.index('Indexing')

for i in range(start, end + 1):
    sorting_concepts[rs_extractor.chapters[i]] = ' '.join(concepts[i])

In [14]:
print('Results using vector store as retriever...')
eval_llm = LangchainLLMWrapper(langchain_llm = ChatOpenAI(temperature = 0))
eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

# for i in range(5):
samples = rs_extractor.validate(sorting_concepts, 
                                    actual_concepts,
                                    metrics = [ResponseRelevancy(), 
                                               LLMContextPrecisionWithReference(), 
                                               Faithfulness(), 
                                               LLMContextRecall(llm = eval_llm),
                                               FactualCorrectness(llm = eval_llm, atomicity = 'low', coverage = 'low'),
                                               SemanticSimilarity()])

Results using vector store as retriever...


Evaluating: 100%|██████████| 24/24 [00:30<00:00,  1.27s/it]

{'answer_relevancy': 0.8548, 'llm_context_precision_with_reference': 1.0000, 'faithfulness': 0.9667, 'context_recall': 1.0000, 'factual_correctness': 0.1275, 'semantic_similarity': 0.8237}
